<a href="https://colab.research.google.com/github/tanvinerkar/ReCAM/blob/master/Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up

Creating run_mlm.py file to train data

Credits: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py

In [ ]:
s='''# coding=utf-8
# Copyright 2020 The HuggingFace Team All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
Fine-tuning the library models for masked language modeling (BERT, ALBERT, RoBERTa...) on a text file or a dataset.
Here is the full list of checkpoints on the hub that can be fine-tuned by this script:
https://huggingface.co/models?filter=masked-lm
"""
# You can also adapt this script on your own masked language modeling task. Pointers for this are left as comments.

import logging
import math
import os
import sys
from dataclasses import dataclass, field
from typing import Optional
from sklearn.metrics import accuracy_score

from datasets import load_dataset

import transformers
from transformers import (
    CONFIG_MAPPING,
    MODEL_FOR_MASKED_LM_MAPPING,
    AutoConfig,
    AutoModelForMaskedLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)
from transformers.trainer_utils import is_main_process


logger = logging.getLogger(__name__)
MODEL_CONFIG_CLASSES = list(MODEL_FOR_MASKED_LM_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc
    }


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
    """

    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The model checkpoint for weights initialization."
            "Don't set if you want to train a model from scratch."
        },
    )
    model_type: Optional[str] = field(
        default=None,
        metadata={"help": "If training from scratch, pass a model type from the list: " + ", ".join(MODEL_TYPES)},
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    train_file: Optional[str] = field(default=None, metadata={"help": "The input training data file (a text file)."})
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file to evaluate the perplexity on (a text file)."},
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    max_seq_length: Optional[int] = field(
        default=None,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated."
        },
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    mlm_probability: float = field(
        default=0.15, metadata={"help": "Ratio of tokens to mask for masked language modeling loss"}
    )
    line_by_line: bool = field(
        default=False,
        metadata={"help": "Whether distinct lines of text in the dataset are to be handled as distinct sequences."},
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": "Whether to pad all samples to `max_seq_length`. "
            "If False, will pad the samples dynamically when batching to the maximum length in the batch."
        },
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["csv", "json", "txt"], "`train_file` should be a csv, a json or a txt file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["csv", "json", "txt"], "`validation_file` should be a csv, a json or a txt file."


def main():
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
    if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
        # If we pass only one argument to the script and it's the path to a json file,
        # let's parse it to get our arguments.
        model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
    else:
        model_args, data_args, training_args = parser.parse_args_into_dataclasses()

    if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty."
            "Use --overwrite_output_dir to overcome."
        )

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if is_main_process(training_args.local_rank) else logging.WARN,
    )

    # Log on each process the small summary:
    logger.warning(
        f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
        + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
    )
    # Set the verbosity to info of the Transformers logger (on main process only):
    if is_main_process(training_args.local_rank):
        transformers.utils.logging.set_verbosity_info()
        transformers.utils.logging.enable_default_handler()
        transformers.utils.logging.enable_explicit_format()
    logger.info("Training/evaluation parameters %s", training_args)

    # Set seed before initializing model.
    set_seed(training_args.seed)

    # Get the datasets: you can either provide your own CSV/JSON/TXT training and evaluation files (see below)
    # or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
    # (the dataset will be downloaded automatically from the datasets Hub
    #
    # For CSV/JSON files, this script will use the column called 'text' or the first column. You can easily tweak this
    # behavior (see below)
    #
    # In distributed training, the load_dataset function guarantee that only one local process can concurrently
    # download the dataset.
    if data_args.dataset_name is not None:
        # Downloading and loading a dataset from the hub.
        datasets = load_dataset(data_args.dataset_name, data_args.dataset_config_name)
    else:
        data_files = {}
        if data_args.train_file is not None:
            data_files["train"] = data_args.train_file
        if data_args.validation_file is not None:
            data_files["validation"] = data_args.validation_file
        extension = data_args.train_file.split(".")[-1]
        if extension == "txt":
            extension = "text"
        datasets = load_dataset(extension, data_files=data_files)
    # See more about loading any type of standard or custom dataset (from files, python dict, pandas DataFrame, etc) at
    # https://huggingface.co/docs/datasets/loading_datasets.html.

    # Load pretrained model and tokenizer
    #
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.
    if model_args.config_name:
        config = AutoConfig.from_pretrained(model_args.config_name, cache_dir=model_args.cache_dir)
    elif model_args.model_name_or_path:
        config = AutoConfig.from_pretrained(model_args.model_name_or_path, cache_dir=model_args.cache_dir)
    else:
        config = CONFIG_MAPPING[model_args.model_type]()
        logger.warning("You are instantiating a new config instance from scratch.")

    if model_args.tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(
            model_args.tokenizer_name, cache_dir=model_args.cache_dir, use_fast=model_args.use_fast_tokenizer
        )
    elif model_args.model_name_or_path:
        tokenizer = AutoTokenizer.from_pretrained(
            model_args.model_name_or_path, cache_dir=model_args.cache_dir, use_fast=model_args.use_fast_tokenizer
        )
    else:
        raise ValueError(
            "You are instantiating a new tokenizer from scratch. This is not supported by this script."
            "You can do it from another script, save it, and load it from here, using --tokenizer_name."
        )

    if model_args.model_name_or_path:
        model = AutoModelForMaskedLM.from_pretrained(
            model_args.model_name_or_path,
            from_tf=bool(".ckpt" in model_args.model_name_or_path),
            config=config,
            cache_dir=model_args.cache_dir,
        )
    else:
        logger.info("Training new model from scratch")
        model = AutoModelForMaskedLM.from_config(config)

    model.resize_token_embeddings(len(tokenizer))

    # Preprocessing the datasets.
    # First we tokenize all the texts.
    if training_args.do_train:
        column_names = datasets["train"].column_names
    else:
        column_names = datasets["validation"].column_names
    text_column_name = "text" if "text" in column_names else column_names[0]

    if data_args.line_by_line:
        # When using line_by_line, we just tokenize each nonempty line.
        padding = "max_length" if data_args.pad_to_max_length else False

        def tokenize_function(examples):
            # Remove empty lines
            examples["text"] = [line for line in examples["text"] if len(line) > 0 and not line.isspace()]
            return tokenizer(
                examples["text"],
                padding=padding,
                truncation=True,
                max_length=data_args.max_seq_length,
                # We use this option because DataCollatorForLanguageModeling (see below) is more efficient when it
                # receives the `special_tokens_mask`.
                return_special_tokens_mask=True,
            )

        tokenized_datasets = datasets.map(
            tokenize_function,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            remove_columns=[text_column_name],
            load_from_cache_file=not data_args.overwrite_cache,
        )
    else:
        # Otherwise, we tokenize every text, then concatenate them together before splitting them in smaller parts.
        # We use `return_special_tokens_mask=True` because DataCollatorForLanguageModeling (see below) is more
        # efficient when it receives the `special_tokens_mask`.
        def tokenize_function(examples):
            return tokenizer(examples[text_column_name], return_special_tokens_mask=True)

        tokenized_datasets = datasets.map(
            tokenize_function,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=not data_args.overwrite_cache,
        )

        if data_args.max_seq_length is None:
            max_seq_length = tokenizer.model_max_length
        else:
            if data_args.max_seq_length > tokenizer.model_max_length:
                logger.warn(
                    f"The max_seq_length passed ({data_args.max_seq_length}) is larger than the maximum length for the"
                    f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}."
                )
            max_seq_length = min(data_args.max_seq_length, tokenizer.model_max_length)

        # Main data processing function that will concatenate all texts from our dataset and generate chunks of
        # max_seq_length.
        def group_texts(examples):
            # Concatenate all texts.
            concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
            total_length = len(concatenated_examples[list(examples.keys())[0]])
            # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
            # customize this part to your needs.
            total_length = (total_length // max_seq_length) * max_seq_length
            # Split by chunks of max_len.
            result = {
                k: [t[i : i + max_seq_length] for i in range(0, total_length, max_seq_length)]
                for k, t in concatenated_examples.items()
            }
            return result

        # Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a
        # remainder for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value
        # might be slower to preprocess.
        #
        # To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
        # https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map
        tokenized_datasets = tokenized_datasets.map(
            group_texts,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            load_from_cache_file=not data_args.overwrite_cache,
        )

    # Data collator
    # This one will take care of randomly masking the tokens.
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=data_args.mlm_probability)

    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"] if training_args.do_train else None,
        eval_dataset=tokenized_datasets["validation"] if training_args.do_eval else None,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    # Training
    if training_args.do_train:
        model_path = (
            model_args.model_name_or_path
            if (model_args.model_name_or_path is not None and os.path.isdir(model_args.model_name_or_path))
            else None
        )
        trainer.train(model_path=model_path)
        trainer.save_model()  # Saves the tokenizer too for easy upload

    # Evaluation
    results = {}
    if training_args.do_eval:
        logger.info("*** Evaluate ***")

        eval_output = trainer.evaluate()

        perplexity = math.exp(eval_output["eval_loss"])
        results["perplexity"] = perplexity

        output_eval_file = os.path.join(training_args.output_dir, "eval_results_mlm.txt")
        if trainer.is_world_process_zero():
            with open(output_eval_file, "w") as writer:
                logger.info("***** Eval results *****")
                for key, value in results.items():
                    logger.info(f"  {key} = {value}")
                    writer.write(f"{key} = {value}")

    return results


def _mp_fn(index):
    # For xla_spawn (TPUs)
    main()


if __name__ == "__main__":
    main()'''


Creating training Python file.

In [ ]:
f=open('run_mlm.py','w')
f.write(s)
f.close()

Installing dependencies.

In [ ]:
!pip install datasets

     |████████████████████████████████| 163kB 7.5MB/s 
     |████████████████████████████████| 17.7MB 202kB/s 
     |████████████████████████████████| 245kB 37.4MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [ ]:
!pip install transformers
!pip install fitbert

     |████████████████████████████████| 1.4MB 8.2MB/s 
     |████████████████████████████████| 890kB 27.4MB/s 
     |████████████████████████████████| 2.9MB 36.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ac0facafd95095e54cd4f774c01571dc46ae9effd3bf3f26bdb40fbb0e2a893c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 225kB 9.7MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 61kB 5.5MB/s 
  Created wheel for fitbert: filename=fitbert-0.9.0-cp36-none-any.whl size=214164 sha256=d0cbc4ec8a7c902b47b6260fd15eec0e0c351fcb4e7768d5d63b7937e45cc144
  Stored in directory: /root/.cache/pip/wheels/96/e2/5b/b847ce6a5fac79014a603441cc446fda9e6f7099da0da8d8b0
  Created wheel for dill: filename=dill-0.2.5-cp36-none-any.whl size=52423 sha256=9cbd7a068e19fa2997ec135564a24

In [ ]:
!pip install dill==0.2.7.1

     |████████████████████████████████| 71kB 5.0MB/s 
  Created wheel for dill: filename=dill-0.2.7.1-cp36-none-any.whl size=54544 sha256=fc8b36c31c48153c4943704e1d5b37ebfb6661368d276ccbb3a37e24da7c3266
  Stored in directory: /root/.cache/pip/wheels/99/c4/ed/1b64d2d5809e60d5a3685530432f6159d6a9959739facb61f2
Successfully built dill
ERROR: pyfunctional 1.2.0 has requirement dill==0.2.5, but you'll have dill 0.2.7.1 which is incompatible.
ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.2.7.1 which is incompatible.
  Found existing installation: dill 0.2.5
    Uninstalling dill-0.2.5:
      Successfully uninstalled dill-0.2.5


In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 71kB 4.7MB/s 
     |████████████████████████████████| 1.3MB 13.0MB/s 
     |████████████████████████████████| 2.9MB 24.4MB/s 
     |████████████████████████████████| 1.1MB 21.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.9-cp36-none-any.whl size=101034 sha256=ecd2549b8a82b8bd7fc1ef02bccad389dfba12826ac7caeb2bf7e1c9438ae678
  Stored in directory: /root/.cache/pip/wheels/fc/89/43/f2f5bc00b03ef9724b0f6254a97eaf159a4c4ddc024b33e07a
Successfully built sentence-transformers
  Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4
  Found existing installation: transformers 4.0.1
    Uninstalling transformers-4.0.1:
      Successfully uninstalled transformers-4.0.1


In [ ]:
import torch
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
import csv
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
stop_words_l=stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import dill
dill.dill._reverse_typemap['ClassType'] = type

In [ ]:
dill.__version__

'0.2.7.1'

Modifying some dependencies.

In [ ]:
f=open('/usr/local/lib/python3.6/dist-packages/datasets/utils/py_utils.py')
r=f.read()
r=r.replace('_dill','dill')
f.close()
f=open('/usr/local/lib/python3.6/dist-packages/datasets/utils/py_utils.py','w')
f.write(r)
f.close()

In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:32<00:00, 12.7MB/s]


# Creating training files

Using cleaned data in csv file to create training data file according to requirement.

In [ ]:
def most_similar(doc_id,similarity_matrix,matrix,df):
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    return similar_ix[1:]
def create_with_context(path, fname, num):
  df = pd.read_csv(path)
  content = []
  for idx, row in df.iterrows():
    ques = row['question']
    passage = row['article']

    ques = ques.replace('@placeholder', row["option_"+str(row["label"])])

    docs = passage.split('. ')
    docs.insert(0, ques)

    temp_df = pd.DataFrame(docs,columns=['sentences'])
    temp_df['cleaned']=temp_df.sentences.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l))
    
    document_embeddings = sbert_model.encode(temp_df['cleaned'])
    pairwise_similarities=cosine_similarity(document_embeddings)

    top_sen = most_similar(0,pairwise_similarities,'Cosine Similarity', temp_df)[:num]

    # len_ques = len(tokenizer.tokenize(row['question']))
    sim_sent = [docs[x] for x in top_sen]
    # len_pass = len(tokenizer.tokenize(' '.join(sim_sent)))

    ques = ' '.join(sim_sent) + ques

    content.append(ques+'\n')
  f = open(fname+'.txt', 'w')
  f.writelines(content)
  f.close()


In [ ]:
paths = {'/content/train_task_1.csv': 'train1c', '/content/dev_task_1.csv': 'dev1c',}
for path in paths:
  create_with_context(path, paths[path], 2)  #adds top 2 similar sentences from context to training file.

# Train MLM

Using the run_mlm.py file for fine-tuning a pre-trained BERT LM Model.

In [ ]:
!python run_mlm.py --model_name_or_path bert-base-cased --train_file '/content/train1c.txt' --validation_file '/content/dev1c.txt' --do_train --do_eval --line_by_line --output_dir /tmp/test-mlm-bert-1c

2020-12-13 16:19:44.197195: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/13/2020 16:19:45 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
12/13/2020 16:19:45 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='/tmp/test-mlm-bert-1c', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Dec13_16-19-45_d50362bc16f6', loggin

# Evaluation function

Function for passing data to model for evaluation.

For each sample in test data, the following functions rank each context sentence by similarity to the question without placeholder. The top 2 sentences with the question and list of options are passed to FitBERT whihc uses the custom model trained above.


In [ ]:
def most_similar(doc_id,similarity_matrix,matrix,df):
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    return similar_ix[1:]

def top_similar(df, num, predname=''):
  preds = np.zeros(df.shape[0])
  pred_labels = np.zeros(df.shape[0])
  for idx, row in df.iterrows():
    ques = row['question']
    passage = row['article']

    ques = ques.replace('@placeholder', '')

    docs = passage.split('. ')
    docs.insert(0, ques)

    temp_df = pd.DataFrame(docs,columns=['sentences'])
    temp_df['cleaned']=temp_df.sentences.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l))
    
    document_embeddings = sbert_model.encode(temp_df['cleaned'])
    pairwise_similarities=cosine_similarity(document_embeddings)

    top_sen = most_similar(0,pairwise_similarities,'Cosine Similarity', temp_df)[:num]

    len_ques = len(tokenizer.tokenize(row['question']))
    sim_sent = [docs[x] for x in top_sen]
    len_pass = len(tokenizer.tokenize(' '.join(sim_sent)))

    masked_string = ' '.join(sim_sent) + row['question']

    options=[]
    for i in range(5):
      options.append(row['option_'+str(i)])
    
    ranked = fb.rank(masked_string, options)

    pred_labels[idx] = options.index(ranked[0])

  print(np.sum(preds)/preds.shape[0])
  np.savetxt(predname+'.txt', pred_labels)
  del preds

Load model trained with context.

In [ ]:
model_path = '/tmp/test-mlm-bert-1c-3'
from transformers import AutoModelForMaskedLM, AutoConfig, AutoTokenizer
config = AutoConfig.from_pretrained(model_path)
model = AutoModelForMaskedLM.from_pretrained(
            model_path,
            config=config,
        )
tokenizer = AutoTokenizer.from_pretrained(
            model_path
        )
from fitbert import FitBert
fb = FitBert(model=model, tokenizer=tokenizer, mask_token='@placeholder')

device: cuda
using custom model: ['BertForMaskedLM']


# Predict labels for test data set.

Clean the test data.

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

def clean_data(data_path, fname):
    data = pd.read_csv(data_path)
    remove_words = ['Mr.', 'Mr', 'Mrs.', 'Mrs', 'said', 'bbc', 'said:', 'said.', 'said,', 'said:"', 'said."', 'said,"', 'we re', 'we ve', 'told', \
                'Media playback is not supported on this device', 'Media playback is not supported on this device.', \
                'Media playback supported device', 'Media playback supported device.', '\'Media playback supported device', \
                'media playback unsupported device', 'Media playback is unsupported on your device', \
                'Follow us on Facebook , on Twitter @BBCNewsEnts , or on Instagram at newsents . If you have a story suggestion email entertainment.news@.co.uk .', \
                'Follow us on Twitter @BBCNewsEnts , on Instagram , or if you have a story suggestion email entertainment.news@.co.uk .', \
                'Follow us on @BBCNewsEnts , on Instagram at newsents , or if you have a story suggestion email entertainment.news@.co.uk', \
                'Follow us on @BBCNewsEnts , on Instagram , or if you have a story suggestion email entertainment.news@.co.uk', \
                'Last updated at', 'Subscribe to the BBC Sport newsletter to get our pick of news , features and video sent to your inbox .']
    for i in remove_words:
        data['article'] = data['article'].str.replace(i, '')
    data['article'] = data['article'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    data.to_csv(fname, index = False)
    
paths = ['/content/task1_test_old.csv', '/content/task2_test_old.csv']
clean_data(paths[0], "/content/task1_test.csv")
clean_data(paths[1], "/content/task2_test.csv")

Modify this to the test data file path.

In [ ]:
test_path = '/content/task1_test.csv'

Saves the predicted labels as a text file.

In [ ]:
df = pd.read_csv(test_path) #reading test data
top_similar(df, 3, 'task1_predicted') #evaluation function

## Task 3 

Task 3 is using the model of Task 1 to predict values on the test data of Task 2.

In [ ]:
test_path = '/content/task2_test.csv'

In [ ]:
df = pd.read_csv(test_path) #reading test data
top_similar(df, 3, 'task1-train-task2-test') #evaluation function

# Evaluation script for TAs

Outputs accuracy by comparing both label files.

True labels file should have one label on each line, with no headers.

Accuracy for Task 1

In [ ]:
pred = 'task1_predicted.txt' # modify this to predicted labels file path
actual = '' # modify this to true labels file path
f1 = open(pred)
predv = np.loadtxt(pred)
f1.close()
f1 = open(actual)
truev = np.loadtxt(actual)
f1.close()

matches = np.equal(predv, truev)
acc = np.sum(matches) / matches.size
print("Accuracy is", acc*100)

Accuracy for Task 3

In [ ]:
pred = 'task1-train-task2-test.txt' # modify this to predicted labels file path
actual = '' # modify this to true labels file path
f1 = open(pred)
predv = np.loadtxt(pred)
f1.close()
f1 = open(actual)
truev = np.loadtxt(actual)
f1.close()

matches = np.equal(predv, truev)
acc = np.sum(matches) / matches.size
print("Accuracy is", acc*100)